1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
2. сделать feature engineering
3. обучить любой классификатор (какой вам нравится)
4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
5. применить random negative sampling для построения классификатора в новых условиях
6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
7. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

In [1]:
import pandas as pd
import requests as r
import numpy as np

from lightgbm  import LGBMClassifier

from sklearn.model_selection import train_test_split

from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score


In [2]:
def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 

In [3]:
data_file = r.get('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data').text
data = pd.DataFrame([i.split(',') for i in data_file[:-1].split('\n')], columns=[
    'sample_code_number',
    'clump_thickness',
    'uniformity_of_cell_size',
    'uniformity_of_cell_shape',
    'marginal_athesion',
    'single_epithelial_cell_size',
    'bare_nuclei',
    'bland_chromatin',
    'normal_nucleoli',
    'mitoses',
    'target'
], dtype='float64')
data.head()

,sample_code_number,clump_thickness,uniformity_of_cell_size,uniformity_of_cell_shape,marginal_athesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,target
0,1000025.0,5.0,1.0,1.0,1.0,2.0,1,3.0,1.0,1.0,2.0
1,1002945.0,5.0,4.0,4.0,5.0,7.0,10,3.0,2.0,1.0,2.0
2,1015425.0,3.0,1.0,1.0,1.0,2.0,2,3.0,1.0,1.0,2.0
3,1016277.0,6.0,8.0,8.0,1.0,3.0,4,3.0,7.0,1.0,2.0
4,1017023.0,4.0,1.0,1.0,3.0,2.0,1,3.0,1.0,1.0,2.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sample_code_number           699 non-null    float64
 1   clump_thickness              699 non-null    float64
 2   uniformity_of_cell_size      699 non-null    float64
 3   uniformity_of_cell_shape     699 non-null    float64
 4   marginal_athesion            699 non-null    float64
 5   single_epithelial_cell_size  699 non-null    float64
 6   bare_nuclei                  699 non-null    object 
 7   bland_chromatin              699 non-null    float64
 8   normal_nucleoli              699 non-null    float64
 9   mitoses                      699 non-null    float64
 10  target                       699 non-null    float64
dtypes: float64(10), object(1)
memory usage: 60.2+ KB


In [5]:
data.bare_nuclei.value_counts()

1     402
10    132
5      30
2      30
3      28
8      21
4      19
?      16
9       9
7       8
6       4
Name: bare_nuclei, dtype: int64

In [6]:
data.bare_nuclei = data.bare_nuclei.apply(lambda x: '5' if x=='?' else x)
data.bare_nuclei.value_counts()

1     402
10    132
5      46
2      30
3      28
8      21
4      19
9       9
7       8
6       4
Name: bare_nuclei, dtype: int64

In [7]:
data.bare_nuclei = data.bare_nuclei.astype('float64')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sample_code_number           699 non-null    float64
 1   clump_thickness              699 non-null    float64
 2   uniformity_of_cell_size      699 non-null    float64
 3   uniformity_of_cell_shape     699 non-null    float64
 4   marginal_athesion            699 non-null    float64
 5   single_epithelial_cell_size  699 non-null    float64
 6   bare_nuclei                  699 non-null    float64
 7   bland_chromatin              699 non-null    float64
 8   normal_nucleoli              699 non-null    float64
 9   mitoses                      699 non-null    float64
 10  target                       699 non-null    float64
dtypes: float64(11)
memory usage: 60.2 KB


In [8]:
data.describe()

,sample_code_number,clump_thickness,uniformity_of_cell_size,uniformity_of_cell_shape,marginal_athesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,target
count,6.990000e+02,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,1.071704e+06,4.417740,3.134478,3.207439,2.806867,3.216023,3.577969,3.437768,2.866953,1.589413,2.689557
std,6.170957e+05,2.815741,3.051459,2.971913,2.855379,2.214300,3.608431,2.438364,3.053634,1.715078,0.951273
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.706885e+05,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


из опсиания мы знаем, что стобец sample_code_number является просто айдишником. Перенесем его в отдельную переменную "индекс", чтобы не мешал обучению.

In [9]:
_idx = data.sample_code_number
data = data.drop('sample_code_number', axis=1)
# data = data.set_index('sample_code_number')
data.head()

,clump_thickness,uniformity_of_cell_size,uniformity_of_cell_shape,marginal_athesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,target
0,5.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,1.0,2.0
1,5.0,4.0,4.0,5.0,7.0,10.0,3.0,2.0,1.0,2.0
2,3.0,1.0,1.0,1.0,2.0,2.0,3.0,1.0,1.0,2.0
3,6.0,8.0,8.0,1.0,3.0,4.0,3.0,7.0,1.0,2.0
4,4.0,1.0,1.0,3.0,2.0,1.0,3.0,1.0,1.0,2.0


Поскольку я выбрал модель LGBM, которая основана на деревьях, скалер нам применять смысла нету. переходим к обучению модели.  Для чистоты эксперимента зададим дефолтные параметры, и укажем дисбаланс целефой переменной.

In [10]:
data.target = data.target.apply(lambda x: 0 if x==2 else 1)
data.iloc[:, -1].value_counts()

0    458
1    241
Name: target, dtype: int64

In [11]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((489, 9), (210, 9), (489,), (210,))

In [13]:
model = LGBMClassifier(is_unbalance=True)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [14]:
evaluate_results(y_test, y_pred)

Classification results:
f1: 97.37%
roc: 98.22%
recall: 98.67%
precision: 96.10%


In [26]:
pos_ind = data.loc[data.target==1].index
unl_ind = data.loc[data.target!=1].index

In [87]:
X_pos = data.iloc[(np.random.choice(pos_ind, size=int(pos_ind.shape[0] / 2), replace=False)).reshape((-1))]
X_unl = data.loc[data.index==0]
X_sample = pd.concat([X_pos, X_unl], ignore_index=True)


In [88]:
model.fit(X_sample.iloc[:,:-1], X_sample.iloc[:,-1])
y_pred = model.predict(X_test)
evaluate_results(y_test, y_pred)

Classification results:
f1: 89.70%
roc: 93.41%
recall: 98.67%
precision: 82.22%


Непонятно, почему метрики не изменились. Оставьте, пожалуйста, комментарии, чтобы я переделал это задание.